In [138]:
import twitter
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import collections
from requests_oauthlib import OAuth1Session

In [139]:
# consumer_key="vSsCY8ZdyOKqI2nBVpLElBKDD"
# consumer_secret="xHWGtoCllCtc7lb5GkCAhJVU4q7TGN8TBtZZOEFqK1Rez8LQ1t"
# def get_resource_token():
#     #create an object of OAuth1Session
#     request_token = OAuth1Session(client_key=consumer_key, client_secret=consumer_secret)
#     # twitter endpoint to get request token
#     url = 'https://api.twitter.com/oauth/request_token'
#     # get request_token_key, request_token_secret and other details
#     data = request_token.get(url)
#     # split the string to get relevant data
#     data_token = str.split(data.text, '&')
#     ro_key = str.split(data_token[0], '=')
#     ro_secret = str.split(data_token[1], '=')
#     resource_owner_key = ro_key[1]
#     resource_owner_secret = ro_secret[1]
#     resource = [resource_owner_key, resource_owner_secret]
#     return resource
#
# resource_token = get_resource_token()

In [140]:
api = twitter.Api(consumer_key="vSsCY8ZdyOKqI2nBVpLElBKDD",
                  consumer_secret="xHWGtoCllCtc7lb5GkCAhJVU4q7TGN8TBtZZOEFqK1Rez8LQ1t",
                  access_token_key="806043476831809537-F2GoVNl8ngMXD0z0fdsvCSJXxolZhiY",
                  access_token_secret="aupIcZVfqSx9xyWYaa2Kyu7PJmfnpdah8G0Urgjwtz1U1"
                  )
# api = twitter.Api(consumer_key=consumer_key,
#                   consumer_secret=consumer_secret,
#                   access_token_key=resource_token[0],
#                   access_token_secret=resource_token[1])
api.sleep_on_rate_limit = True

In [172]:
def getUserFromName(name, api):
    user_list = api.GetUsersSearch(term=name, page=1, count=1)
    if user_list:
        user = user_list[0]
        if user.verified and (not user.protected):
            return user
    return None

In [173]:
queries = []
# Uncomment these lines to read the list of names
with open('./names.csv', 'r') as f:
    queries = f.read().split('\n')[:-1]

In [143]:
# dummy_query_list
# queries = ["Luis Suarez", "Sergio Busquets", "Iker Casillas", "Malena Ortiz Cruz", "Diego Lopez",
#            "Isabel Garzo", "Marc ter Stegan", "Samuel Umtiti", "Andres Iniesta", "Arturo Vidal",
#            "Jordi Alba", "Sergi Roberto", "Rafa Nadal", "Roger Federer", "Gerard Pique",
#            "Antoine Griezmann", "marie portolano"]

In [174]:
# For now saving all kinds of graphs... may need only one later
g = nx.Graph()
dg = nx.DiGraph()
id_to_name = {}
for i in range(len(queries)):
    user = getUserFromName(queries[i], api)

    if user:
        # row = [repr(user.id), user.name, user.screen_name]
        print(f'{i+1}', end=" ")
        id_to_name[user.id] = user.name
        g.add_node(user.name, id=repr(user.id), screen_name=user.screen_name)
        dg.add_node(user.name, id=repr(user.id), screen_name=user.screen_name)
        queries[i] = user.name

1 2 3 4 5 6 7 9 11 12 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 33 35 36 37 38 39 40 41 43 44 45 46 47 48 49 51 52 53 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 77 78 79 80 81 82 83 84 85 86 89 91 94 95 96 97 98 99 100 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 131 132 133 134 135 136 137 138 139 140 141 142 147 148 149 150 154 158 159 160 161 163 164 167 168 170 172 173 174 176 178 181 182 184 185 186 188 189 190 191 193 196 197 198 199 200 201 202 204 206 208 209 210 213 215 216 219 220 222 223 224 225 226 228 229 230 232 233 234 235 239 240 241 242 246 249 250 251 252 254 255 256 258 259 260 261 265 266 268 269 270 271 274 275 276 277 278 280 281 282 285 286 289 290 291 293 294 295 296 298 299 300 301 302 305 306 308 311 315 316 317 318 319 321 323 324 327 329 330 331 332 333 334 335 336 337 338 339 341 342 344 345 346 

In [176]:
for query_name in queries:

    if not g.has_node(query_name):
        print(f"Skipping {query_name}")
        continue

    query_id = g.nodes[query_name]['id']
    friendIDs = api.GetFriendIDs(query_id)
    connection_count = 0
    for friendID in friendIDs:
        if friendID in id_to_name.keys():

            connection_count += 1
            friend_name = id_to_name[friendID]
            # print(f'{query_name} is following {friend_name}')
            dg.add_edge(query_name, friend_name)

            # 2 nodes in an undirected graph are connected if they both follow each other
            if dg.has_edge(friend_name, query_name):
                g.add_edge(friend_name, query_name)

    print(f'{connection_count}', end = " ")

39 41 51 14 14 1 21 Skipping michael thomas wr
18 Skipping clyde edwards helaire
13 23 Skipping julio jones
32 1 60 63 25 35 2 36 47 8 50 18 9 27 18 20 30 50 Skipping dj moore wr
42 Skipping amari cooper
61 37 49 18 15 24 10 Skipping david johnson rb
45 17 28 25 39 14 13 Skipping leveon bell
8 45 2 Skipping darren waller
16 15 49 29 18 15 11 20 38 11 14 15 16 13 3 23 10 23 11 23 10 Skipping jordan howard
7 20 13 4 22 16 26 8 17 12 Skipping matthew stafford
Skipping deebo samuel
9 Skipping antonio gibson
6 Skipping jamison crowder
Skipping john brown
12 19 24 24 9 9 6 Skipping ronald jones ii
0 10 8 21 14 26 20 6 10 4 16 2 15 7 25 6 29 6 13 3 28 23 47 10 32 6 16 39 Skipping mike williams wr
2 13 5 12 16 79 36 12 9 8 11 4 Skipping jack doyle
Skipping san francisco defense
Skipping jalen reagor
Skipping pittsburgh defense
13 9 5 7 Skipping baltimore defense
Skipping brandon aiyuk
Skipping tony pollard
24 Skipping justin jefferson
Skipping philip rivers
Skipping buffalo defense
6 49 31 8 S

In [177]:
%matplotlib qt
plt.plot()
nx.draw(dg, with_labels=True)
plt.show()

/home/aman/anaconda3/envs/ml/lib/python3.7/site-packages/matplotlib/backends/backend_agg.py:214: RuntimeWarning: Glyph 57614 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/aman/anaconda3/envs/ml/lib/python3.7/site-packages/matplotlib/backends/backend_agg.py:214: RuntimeWarning: Glyph 57661 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/aman/anaconda3/envs/ml/lib/python3.7/site-packages/matplotlib/backends/backend_agg.py:214: RuntimeWarning: Glyph 57361 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/aman/anaconda3/envs/ml/lib/python3.7/site-packages/matplotlib/backends/backend_agg.py:183: RuntimeWarning: Glyph 57614 missing from current font.
  font.set_text(s, 0, flags=flags)
/home/aman/anaconda3/envs/ml/lib/python3.7/site-packages/matplotlib/backends/backend_agg.py:183: RuntimeWarning: Glyph 57661 missing from current font.
  font.set_text(s, 0, flags=flags)
/home/aman/anaconda3/envs/ml/lib/python3.7/site-packages/ma

In [178]:
list(nx.connected_components(g))

[{'$.',
  '.',
  '7⃣ Leonard Fournette',
  'AJ Brown',
  'AJ “The Sauce” Dillon',
  'Aaron Jones',
  'Aaron Rodgers',
  'Adam Humphries',
  'Adam Thielen',
  'Aj Green',
  'Alexander Mattison',
  'Allen Lazard',
  'Allen Robinson II',
  'Alshon Jeffery',
  'Alvin Kamara',
  'Anthony Miller',
  'Auden Tate',
  'Austin Ekeler',
  'Austin M Hooper',
  'Baker Mayfield',
  'BigBen7.com',
  'Blake Jarwin',
  'Booger',
  'Boston Scott',
  'Brandin Cooks',
  'Brandon McManus',
  'Breshad Perriman',
  'Brett Maher',
  'Brian Hill',
  'Bryan Edwards',
  'CALVIN RIDLEY',
  'Cam akers',
  'Cameron 1 Newton',
  'Carlos Hyde',
  'Carson Wentz',
  'CeeDee Lamb',
  'Chase Claypool',
  'Chase Edmonds',
  'Chief David Njoku',
  'Chris Carson',
  'Chris Godwin',
  'Chris Thompson',
  'Christian Kirk',
  'Christian McCaffrey',
  'Christopher Herndon IV',
  'Cole Beasley',
  'Cooper Kupp',
  'Corey Davis',
  'Courtland Sutton♠️',
  'Curtis Samuel',
  'DARWIN THOMPSON',
  'DIGGS',
  'DJ Chark',
  'DK Metcal

In [179]:
# api.GetSearch(raw_query="screen_name=fundacionmessi")
# len(api.GetFriends(2917808716))

100

In [36]:
# api.GetFriendIDs(2917808716)[0]

95257207

In [180]:
with open('./twitter_graph', 'wb') as f:
    pickle.dump(g, f)

with open('./twitter_directed_graph', 'wb') as f:
    pickle.dump(dg, f)

In [181]:
with open('./twitter_graph', 'rb') as f:
    new_g = pickle.load(f)

with open('./twitter_directed_graph', 'rb') as f:
    new_dg = pickle.load(f)

In [182]:

plt.plot()
nx.draw(new_g, with_labels=True)
plt.show()

In [183]:
def draw_graph(G):
    degree_sequence = sorted([d for n, d in G.degree()], reverse=True)  # degree sequence
    degreeCount = collections.Counter(degree_sequence)
    deg, cnt = zip(*degreeCount.items())

    fig, ax = plt.subplots()
    plt.bar(deg, cnt, width=0.80, color="b")

    plt.title("Degree Histogram")
    plt.ylabel("Count")
    plt.xlabel("Degree")
    ax.set_xticks([d + 0.4 for d in deg])
    ax.set_xticklabels(deg)

    # draw graph in inset
    plt.axes([0.4, 0.4, 0.5, 0.5])
    pos = nx.spring_layout(G)
    plt.axis("off")
    nx.draw_networkx_nodes(G, pos, node_size=10)
    nx.draw_networkx_edges(G, pos, alpha=0.4)
    plt.show()

In [184]:
draw_graph(g)

In [186]:
# temp code. Might need later
for n, nbrsdict in g.adjacency():
    print(n, end="\t")
    print(list(nbrsdict.keys()))
    # for nbr, edge in nbrsdict.items():
    #     print(n, nbr)

Christian McCaffrey	['Saquon Barkley', 'Ezekiel Elliott', '\ue10eDerrick Henry \ue13d\ue011', 'Dalvin Cook', 'Nick Chubb', 'George Kittle', 'Austin Ekeler', 'Mike Evans', 'Mark Andrews 8⃣9️⃣', 'James Conner', 'JuJu Smith-Schuster', 'Cooper Kupp', 'Tyler Lockett', 'Zach Ertz', 'Mark Ingram II', 'Keenan Allen', 'Michael Gallup', 'Jk dobbins', 'Cameron 1 Newton', 'Matt Breida', 'Sony Michel', 'Austin M Hooper', 'Baker Mayfield', 'Eric Ebron', 'Allen Lazard', 'Curtis Samuel', 'Teddy Bridgewater', 'Harrison Butker', 'OJ Howard ™', 'Greg Olsen', 'Cole Beasley', 'Chief David Njoku', 'LeSean Shady Mccoy', 'Wayne Gallman II']
Saquon Barkley	['Christian McCaffrey', 'Ezekiel Elliott', 'Miles Sanders', 'George Kittle', 'Chris Godwin', 'Mike Evans', 'Lamar Jackson', 'James Conner', 'Odell Beckham Jr', 'Todd Gurley II', 'Jonathan Taylor', 'DK Metcalf', 'Mark Ingram II', 'DIGGS', 'Cam akers', 'Jarvis Juice Landry', '7⃣ Leonard Fournette', 'Evan Engram', 'Julian Edelman', 'SWIFT7️⃣', 'Jk dobbins', 'To

In [185]:
len(g.nodes)


257